# Interacting with Ethereum using web3.py and Jupyter Notebooks
Step by step guide for setting up a Jupyter notebook, connecting to an Ethereum node and working with a Smart Contract.

In this tutorial we are using Python 3, so make sure that **python** and **pip** are versioned correctly.
<hr>

## STEP 0: Getting tutorial materials

Grab a copy of the files that we use in this tutorial:

+ Using Git:

    <code>git clone https://github.com/apguerrera/ethereum-notebooks.git</code>


+ Or download it manually from https://github.com/apguerrera/ethereum-notebooks

<hr>

## STEP 1: Installing dependencies
+ Install [Jupyter](https://jupyter.org/)

    <code>pip install --upgrade pip</code>

    <code>pip install jupyter</code>


+ Install [Web3.py](https://web3py.readthedocs.io/en/stable/), Python module for accessing Ethereum blockchain

    <code>pip install web3</code>


+ Install [py-solc-x](https://pypi.org/project/py-solc-x/), Python module for compiling Solidity contracts

    We use **py-solc-x** instead of **py-solc** to compile contracts, since py-solc doesn't support Solidity versions v.0.5.x.
    
    Also **py-solc-x** provides an ability to choose between different Solidity compiler versions.
    
    <code>pip install py-solc-x</code>
    
    Note: the module itself doesn't contain **solc** executable, so let's install solc executable version 0.5.3 that we use in this tutorial
    
    <code>python -m solcx.install v0.5.3</code>


+ To install Geth go to https://ethereum.org/cli and follow the instructions

<hr>

In [11]:
!pip install --upgrade pip
!pip install web3
!pip install py-solc-x

Requirement already up-to-date: pip in /Users/adrian/anaconda3/lib/python3.6/site-packages (19.0.3)


## STEP 2: Running local Geth node

+ Go to the project directory and run in your terminal:

 <code>./00_runGeth.sh</code> 


+ Or use  

<code>geth --dev --dev.period 2 --datadir ./testchain --rpc --rpccorsdomain ‘*’ --rpcport 8646 --rpcapi “eth,net,web3,debug” --port 32323 --maxpeers 0 console</code> 

script which is doing exactly the same


<hr>

## STEP 3: Running Jupyter notebook

**If you're already viewing this notebook in Jupyter live mode, just skip this step.**

+ Open Jupyter notebooks by running the following in your terminal:

    <code>./01_runNotebook.sh</code>
    

+ If you see an error message, try:

    <code>jupyter-notebook</code>

It will open up a window in your browser. Here you need to go to the project folder and open <code>EthereumNotebookNew.ipynb</code>

<hr>

## STEP 4: Conecting to Web3
Web3 has a provider type that lets you connect to a local Ethereum node or endpoint such as [Infura](https://infura.io/).

In our example, we’ll be connecting to a local Geth node running from the /testchain directory, but can be set to any Ethereum node that web3 can connect to.

In [12]:
from web3 import Web3
w3 = Web3(Web3.IPCProvider('./testchain/geth.ipc'))
w3.isConnected()  # if it's false, something went wrong

True

In [13]:
# check that all accounts were pulled from ./testchain directory successfuly
w3.eth.accounts

['0xa00Af22D07c87d96EeeB0Ed583f8F6AC7812827E',
 '0xa11AAE29840fBb5c86E6fd4cF809EBA183AEf433',
 '0xa22AB8A9D641CE77e06D98b7D7065d324D3d6976',
 '0xa33a6c312D9aD0E0F2E95541BeED0Cc081621fd0',
 '0xa44a08d3F6933c69212114bb66E2Df1813651844',
 '0xa55A151Eb00fded1634D27D1127b4bE4627079EA',
 '0xa66a85ede0CBE03694AA9d9dE0BB19c99ff55bD9',
 '0xa77A2b9D4B1c010A22A7c565Dc418cef683DbceC',
 '0xA88A05d2b88283ce84C8325760B72a64591279a2',
 '0xa99A0Ae3354c06B1459fd441a32a3F71005D7Da0',
 '0xAAAA9De1E6C564446EBCA0fd102D8Bd92093c756',
 '0xaBBa43E7594E3B76afB157989e93c6621497FD4b',
 '0xacCa534c9f62Ab495bd986e002DdF0f054caAE4f',
 '0xAddA9B762A00FF12711113bfDc36958B73d7F915',
 '0xaeEa63B5479B50F79583ec49DACdcf86DDEff392',
 '0xaFfa4D3A80Add8CE4018540e056DACb649589394',
 '0xB00bfdE102270687324F9205b693859df64F8923',
 '0xB11Be1D4EF8E94d01cB2695092A79d139A8DAD98',
 '0xB22BE2D9eEF0d7E260CF96A64FEEa0B95ED3E74f',
 '0xB33B7ecf5E47BE3981c74d989d3Af8b665b4B649',
 '0xB44B43d59b738B088b690aE276C1E979aBa8268D',
 '0xb55B57D11

## STEP 5: Compiling contracts with py-solc-x

In [17]:
# compile contract using solcx and return contract interface
# arguments are filepath to the contract and name of the contract
import solc

def compile_contract(path, name):
    compiled_contacts = solc.compile_files([path]) --optimize --pretty-json --combined-json abi,bin,interface
    contract_interface = compiled_contacts['{}:{}'.format(path, name)]
    return contract_interface

contract_path = './contracts/DeepyrsDividendETHToken.sol'
contract_name = 'DividendToken'
contract_interface = compile_contract(contract_path, contract_name)
print(contract_interface)

SolcError: An error occurred during execution
> command: `solc --combined-json abi,asm,ast,bin,bin-runtime,clone-bin,devdoc,interface,opcodes,userdoc ./contracts/DeepyrsDividendETHToken.sol`
> return code: `1`
> stderr:

> stdout:
Invalid option to --combined-json: clone-bin

In [16]:
# check that py-solc-x and solc are installed correctly
import solcx

solcx.get_installed_solc_versions()

ValueError: solc version must be in the format v0.x.x

## STEP 6: Deploying a contract to blockchain
In next steps we'll be using some functions from [/scripts/util.py](https://github.com/apguerrera/ethereum-notebooks/blob/master/scripts/util.py) and [/scripts/whitelist.py](https://github.com/apguerrera/ethereum-notebooks/blob/master/scripts/whitelist.py). It's **highly recommended** to check out this Python files to have better understanding of next steps.
Also we will pass **w3** instance as an argument to imported functions. We don't use **w3** as global variable since it's possible to have different endpoints thus having more than one w3 object in your program.

In [ ]:
# import function that decrypts keystore file and returns account object
# check out tutorial directory in /scripts/util.py
from scripts.util import account_from_key

# compile contract, deploy it from account specified, then return transaction hash and contract interface
def deploy_contract(w3, account, path, name):
    contract_interface = compile_contract(path, name)
    contract = w3.eth.contract(abi=contract_interface['abi'], bytecode=contract_interface['bin'])
    transaction = contract.constructor().buildTransaction({
        'nonce': w3.eth.getTransactionCount(account.address),
        'from': account.address
    })
    signed_transaction = w3.eth.account.signTransaction(transaction, account.privateKey)
    tx_hash = w3.eth.sendRawTransaction(signed_transaction.rawTransaction)
    return tx_hash.hex(), contract_interface

key_path = './testchain/keystore/UTC--2017-05-20T02-37-30.360937280Z--a00af22d07c87d96eeeb0ed583f8f6ac7812827e'
key_passphrase = ''  # empty password for test keystore file, never do that in real life
account = account_from_key(w3, key_path, key_passphrase)

tx_hash, contract_interface = deploy_contract(w3, account, './contracts/WhiteList.sol', 'WhiteList')
tx_hash

Note: **deploy_contract doesn't return the address of created contract**, it returns hash of transaction made to create the contract

To get the address of the contract:

In [ ]:
# import function that waits for deploy transaction to be included to block, and returns address of created contract
# check out tutorial directory in /scripts/util.py
from scripts.util import wait_contract_address

contract_address = wait_contract_address(w3, tx_hash)
contract_address

## STEP 7: Interacting with the contract

In [ ]:
# import function that returns contract object using its address and ABI
# check out tutorial directory in /scripts/util.py
from scripts.util import get_contract

contract = get_contract(w3, contract_address, contract_interface['abi'])
contract.all_functions()  # get all available functions of the contract

In [ ]:
# check out /scripts/util.py and /scripts/whitelist.py
from scripts.whitelist import add_to_list
from scripts.util import wait_event

address_to_add = w3.eth.accounts[17]
tx_hash = add_to_list(w3, account, contract, [address_to_add])
event_added = wait_event(w3, contract, tx_hash, 'AccountListed')
if event_added:
    print(event_added[0]['args'])

In [ ]:
# check out /scripts/whitelist.py
from scripts.whitelist import is_in_list

is_in_list(account, contract, address_to_add)  # check if address in whitelist

## Moving forward
Now you know how to compile Solidity contracts using **solc** and **py-solc-x**, deploy contracts using **Web3** and interact with them!

To see other code snippets and related information please check out [tutorial's GitHub repo](https://github.com/apguerrera/ethereum-notebooks/) and **WhitelistExample** notebook.